# Czyszczenie danych i przygotowanie do analizy
Celem tego sprawozdania jest oczyszczenie i przygotowanie dostarczonego zestawu danych o **pasażerach statku Titanic**. 
Głównym zadaniem jest przekształcenie tych danych z nieuporządkowanych, wybrakowanych lub niepoprawnych w dane przydatne do analizy oraz dla modeli uczenia się maszynowego.

*Aby rozpocząć pracę nad zbiorem danych musimy zaimportować podstawowe biblioteki z których będziemy korzystać tworząc funkcjonalności. <br>
W poniższym sprawozdaniu użyto biblioteki math, numpy oraz pandas.*

In [234]:
import math
import numpy as np
import pandas as pd
import locale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

'en_US.UTF-8'

Następnym krokiem jest **wczytanie pliku** oraz sprawdzenie czy został on poprawnie zaimportowany do notatnika. <br> W dodatku  w komendzie otwierającej definiujemy separator jako znak tabulacji (wynika to z formatu pliku .tsv), a jako kolumnę indeksującą usawiamy **PassengerID**.

In [235]:
titanic_df = pd.read_csv('TitanicMess.tsv', sep='\t', index_col='PassengerId') #wczytanie
titanic_df.head() #wyświetlenie wyników 

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


Nastepnym etapem bedzie **sprawdzenie danych pod katem błednych lub pustych wartości.**

In [236]:
titanic_df.dtypes #sprawdzenie typów kolumn

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object

Jak mozemy zauważyc z powyżej tabeli, kolumna Age,  Fare jest traktowana jako **object**, ze względu przeznaczenie kolumn powinny one zawierać tylko liczby. Prawdopodobnie w tych dwóch kolumnach występują niepoprawne dane, które wpłyneły na złe przypisanie typu kolumny. 

In [237]:
titanic_df.isnull().sum() #podliczenie liczby pustych danych

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

Jak możemy zobaczyć, posiadamy dużą liczbę pustych wartości w kolumnach *Age, Cabin* oraz w *Embarked*. 

Analizujac dane i zawartość kolumn, mozemy zauwazyc ze kolumna **Ship** dla wszystkich rekordów ma taką sama wartość - *Titanic*. Biorąc pod uwagę, że przygotujemy dane dotyczące wyłącznie pasażerów statku Titanic, dlatego możemy tą kolumnę usunąć by nie powodowała nadmiernych obliczeń, jest bezużyteczna. <br> Usuwanie kolumny jest możliwe za pomocą metody *drop()*.

In [238]:
titanic_clear_columns = titanic_df.drop(columns='ship') #usunięcie kolumny
titanic_clear_columns.head() # wyswietlenie bez ship

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S


Po pierwszym kroku oczyszczeniu danych, warto przyjrzeć się pozostałym kolumną i ich zawartością. <br>
**Survived** - kolumna jest cechą wyjściową, zawiera wartości dla każdej obserwacji albo 1 jeśli pasażer przeżył katastrofę albo 0 jeśli zginął. <br>
**Pclass** - kolumna ta zawiera klasę każdego pasażera na statku, są to wartości 1, 2 lub 3. <br>
**Age** - kolumna ta zawiera wiek każdego pasażera na statku, i jest użyteczna, używając pewnych poprawek możemy wydobyć nową cechę danych - dzięki niej możemy określić czy pasażer jest dzieckiem, albo nie (tutaj przyda się również kolumna **Sex**).

Jak wiemy z analizy pustych wartości, kolumna **Age** zawiera pewne zerowe/puste wartości, teoretycznie oczywiście możemy wypełnić te wartości. Używając niektórych statystyk, możemy założyć, że kolumna ta podąża za jakimś nieznanym rozkładem, a większość danych jest powtarzana jako niektóre zjawiska, zwane średnią rozkładu.


Podczas uzupełniania danych, zdecydowałam się na wypełnienie brakujących wartości najbardziej powtarzalną wartością.  Mówiąc najbardziej powtarzalną, mam na myśli średnią wartość kolumny.<br>
Dzięki bibliotece PANDAS, obliczenie średniej jest bardzo łatwe i możemy skozystać z metody mean()

In [239]:
titanic_clear_columns['Age'].mean()

TypeError: can only concatenate str (not "int") to str

Wywołanie metody mean() na kolumnie powoduje bląd. Jest on spowodowany faktem, iż kolumna **Age** jest traktowana jak obiekt, a operacja mean() wymaga od danych typu numerycznego. <br> Kolejnym krokiem bedzie usunięcie błędnych wartości by móc jeszcze raz obliczyć średnią wieku pasażerów. <br>
Na początku wyświetlmy błędne dane, wychwytując je blokiem try-catch oraz podsumujmy ilość błędnych wyników.

In [243]:
wrongAges=0 ## zmienna przechowująca liczbę błednych wyników
for index, row in titanic_clear_columns.iterrows():
    try:
        float(row['Age'])  ## próba parsowania do typu float
    except ValueError:
        wrongAges = wrongAges + 1
        print("Błąd wieku dla passegnerId " + str(index)) ## wyświetlenie błędnego wiersza
    
print("Ilość wyników niebędąca liczbą " + str(wrongAges))        

Błąd wieku dla passegnerId 58
Błąd wieku dla passegnerId 79
Błąd wieku dla passegnerId 112
Błąd wieku dla passegnerId 117
Błąd wieku dla passegnerId 123
Błąd wieku dla passegnerId 124
Błąd wieku dla passegnerId 149
Błąd wieku dla passegnerId 153
Błąd wieku dla passegnerId 154
Błąd wieku dla passegnerId 204
Błąd wieku dla passegnerId 228
Błąd wieku dla passegnerId 297
Błąd wieku dla passegnerId 306
Błąd wieku dla passegnerId 332
Błąd wieku dla passegnerId 470
Błąd wieku dla passegnerId 526
Błąd wieku dla passegnerId 645
Błąd wieku dla passegnerId 677
Błąd wieku dla passegnerId 736
Błąd wieku dla passegnerId 756
Błąd wieku dla passegnerId 768
Błąd wieku dla passegnerId 804
Błąd wieku dla passegnerId 815
Błąd wieku dla passegnerId 832
Błąd wieku dla passegnerId 844
Ilość wyników niebędąca liczbą 25


Następny, krokiem jest próba przekonwertowania wartości do typu float oraz ustawienie poprawnego zakresu dla danych Age. <br> W tym celu ustaliłam zakres wieku od 0 do 123 (ponieważ najstarsza żyjąca na świecie osoba miała prawie 123 lata). W dodatku usunęłam dane niepoprawne zastępując je na tym etapie wartościami **NaN**.
Na koniec sprawdziłam, czy operacja się powiodła sprawdzając jeszcze raz typy kolumn. 

In [244]:
for i, row in titanic_clear_columns.iterrows():
    try:
        if float(row["Age"]) < 0 or float(row["Age"]) > 123:
            titanic_clear_columns.at[i, "Age"] = np.NaN
        else:
            titanic_clear_columns.at[i, "Age"] = float(row["Age"])
        
    except ValueError:
        titanic_clear_columns.at[i, "Age"] = np.NaN
for index, row in titanic_clear_columns.iterrows():
    print(row["Age"])

titanic_clear_columns[["Age"]] = titanic_clear_columns[["Age"]].astype(float).round(1) ## konwersja do float
titanic_clear_columns.dtypes ## sprawdzenie aktualnych typów kolumn


22.0
38.0
26.0
35.0
35.0
nan
54.0
2.0
27.0
14.0
4.0
58.0
20.0
4.0
14.0
55.0
2.0
nan
31.0
nan
35.0
34.0
15.0
4.0
8.0
38.0
nan
19.0
nan
nan
40.0
0.9
nan
66.0
28.0
42.0
nan
21.0
18.0
14.0
40.0
27.0
nan
3.0
19.0
nan
nan
nan
nan
18.0
7.0
21.0
49.0
29.0
65.0
nan
21.0
nan
5.0
11.0
22.0
38.0
45.0
4.0
nan
nan
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
nan
nan
nan
30.0
22.0
29.0
nan
28.0
17.0
33.0
16.0
nan
23.0
24.0
29.0
20.0
46.0
26.0
59.0
nan
71.0
23.0
34.0
34.0
28.0
nan
21.0
33.0
37.0
28.0
21.0
nan
38.0
nan
47.0
nan
22.0
20.0
17.0
21.0
nan
29.0
24.0
2.0
21.0
nan
nan
nan
54.0
12.0
nan
24.0
nan
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
nan
22.0
24.0
19.0
18.0
19.0
27.0
9.0
nan
42.0
51.0
22.0
nan
nan
nan
51.0
16.0
30.0
nan
nan
44.0
40.0
26.0
17.0
1.0
9.0
nan
45.0
nan
28.0
61.0
4.0
1.0
21.0
56.0
18.0
nan
50.0
30.0
36.0
nan
nan
9.0
1.0
4.0
nan
nan
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
nan
42.0
nan
24.0
28.0
nan
34.0
nan
18.0
2.0
32.0
26.0
16.0
40.0
24.0
35.0
22.0
30.0
nan

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare         object
Cabin        object
Embarked     object
dtype: object

Na tym etapie błąd został usunięty, a typ kolumny Age to float64. <br> Ponownie próba obliczenia średniej kolumny Age kończy się sukcesem. 

In [245]:
round(titanic_clear_columns['Age'].mean())

30

Skoro obliczyliśmy już średnią wieku wynoszącą 30 lat, pora podstawić tą wartość pod wartości błędne i puste. 

In [196]:
titanic_clear_columns['Age'] = titanic_clear_columns['Age'].fillna(round(titanic_clear_columns['Age'].mean()))
for index, row in titanic_clear_columns.iterrows():
    print(row["Age"])


22.0
38.0
26.0
35.0
35.0
30.0
54.0
2.0
27.0
14.0
4.0
58.0
20.0
4.0
14.0
55.0
2.0
30.0
31.0
30.0
35.0
34.0
15.0
4.0
8.0
38.0
30.0
19.0
30.0
30.0
40.0
0.9
30.0
66.0
28.0
42.0
30.0
21.0
18.0
14.0
40.0
27.0
30.0
3.0
19.0
30.0
30.0
30.0
30.0
18.0
7.0
21.0
49.0
29.0
65.0
30.0
21.0
30.0
5.0
11.0
22.0
38.0
45.0
4.0
30.0
30.0
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
30.0
30.0
30.0
30.0
22.0
29.0
30.0
28.0
17.0
33.0
16.0
30.0
23.0
24.0
29.0
20.0
46.0
26.0
59.0
30.0
71.0
23.0
34.0
34.0
28.0
30.0
21.0
33.0
37.0
28.0
21.0
30.0
38.0
30.0
47.0
30.0
22.0
20.0
17.0
21.0
30.0
29.0
24.0
2.0
21.0
30.0
30.0
30.0
54.0
12.0
30.0
24.0
30.0
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
30.0
22.0
24.0
19.0
18.0
19.0
27.0
9.0
30.0
42.0
51.0
22.0
30.0
30.0
30.0
51.0
16.0
30.0
30.0
30.0
44.0
40.0
26.0
17.0
1.0
9.0
30.0
45.0
30.0
28.0
61.0
4.0
1.0
21.0
56.0
18.0
30.0
50.0
30.0
36.0
30.0
30.0
9.0
1.0
4.0
30.0
30.0
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
30.0
42.0
30.0
24.0
28.0
30.0
34.0
30.0
1

Podczas analizy możemy zauważyć, że w analizie zestawu danych o pasażerach Titanica, kolumna **Cabin** nie daje nam żadnej użytecznej wiedzy, poza tym najczęściej są to wartości zerowe. Dlatego też nie będziemy jej używać, ponieważ jest bezużyteczna i będzie miała wpływ na każdy model uczenia się maszynowego. Kolejnym etap przetwarzania danych jest usunięcie kolumny z ramki danych.

In [197]:
titanic_clear_columns.drop('Cabin',axis=1, inplace=True) #usunięcie
titanic_clear_columns.head() ## wyświetlenie

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",S


Jak widzieliśmy kolumna **Embarked** zawierała aż 2 puste wartości. <br> Podczas uzupełniania tej kolumny losowo wybrałam S i podstanowiłam uzupełnić brakujące pola. 

In [246]:
titanic_clear_columns['Embarked'] = titanic_clear_columns['Embarked'].fillna('S')
titanic_clear_columns.isnull().values.any()

True

Na tym etapie czyszczenia wszystkie puste wartośći zostały uzupełnione, o czym świadczy wynik metody  *titanic_clear_columns.isnull().values.any()*

Kolejnym etapem jest sprawdzenie wartości przechowywanych w kolumnie **Sex**. Pomimo obecnych trendów określania się jako osoby niebinarne postanowiłam założyć, że kolumnie tej (dotyczącej pasażerów żyjących na początku XX w.) wszystkie osoby mogły zadeklarować tylko jedną z dwóch płci: MALE, FEMALE.


In [247]:
unkalna_plec = titanic_clear_columns['Sex'].unique()
print(unkalna_plec)

['male' 'female' 'malef' 'mal' 'fem' 'femmale']


Niestety jak możemy zauważyć, kolumna ta przechowuje aż 6 wartości. Prawdopodbnie są one zwykłymi literówkami. Postanowiłam je zamienić według schematu <br>
**malef** -> *male* <br>
**male** -> *male* <br>
**fem** -> *female* <br>
**femmale** -> *female* <br>

In [217]:
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['malef'],'male')
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['mal'],'male')
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['fem'],'female')
titanic_clear_columns['Sex'] = titanic_clear_columns['Sex'].replace(['femmale'],'female')
unkalna_plec = titanic_clear_columns['Sex'].unique()
print(unkalna_plec) # sprawdzenie czy operacja sie powiodła

['male' 'female']


Dzięki metodzie unique() sprawdziliśmy, czy operacja powiodła się. Na tym etapie w kolumnie **Sex** znajdują się już tylko 2 możliwe wartości.

W poprzedniej analizie typów mogliśmy zauwazyć, że kolumna **Fare**, która przechowuje wartości ceny biletu jest traktowana jako Obiekt, zdecydowałam się na przekonwertowanie ich do typu float *podobnie jak w przypadku Age* oraz zaokrąglenie do 2 miejsc po przecinku. <br>
Pod koniec wyświetlone zostały typy kolumn po tej modyfikacji. 

In [219]:

for i, row in titanic_clear_columns.iterrows():
    try:
        titanic_clear_columns.at[i, 'Fare'] = float(row['Fare'])       
    except ValueError:
        titanic_clear_columns.at[i, 'Fare'] = np.NaN

titanic_clear_columns[['Fare']] = titanic_clear_columns[['Fare']].astype(float).round(2)
titanic_clear_columns.dtypes


Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked     object
dtype: object

Jak widać kolumna **Fare** jest traktowana jako zmienna liczbowa. 
Kolejnym krokiem będzie sprawdzenie występujących duplikatów.

In [228]:
titanic_clear_columns.duplicated(keep='first')

PassengerId
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
11       True
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
11       True
25      False
26      False
27      False
28      False
29      False
30      False
31      False
32      False
33      False
34      False
35      False
36      False
37      False
38      False
39      False
40      False
41      False
42      False
43      False
44      False
45      False
46      False
47      False
48      False
49      False
50      False
51      False
52      False
53      False
54      False
55      False
56      False
57      False
58      False
59      False
60      False
61      False
62      False
63      False
64      False
65      False
66      False
67      False
68      False
69      False
70      False
71      

Jak możemy zauważyć niektóre rekordy (*dla PassengerId=11*) potwarzają się. Następnym krokiem bedzie usunięcie duplikatów.

In [230]:
titanic_clear_columns.drop_duplicates(keep = False, inplace = True) ## usunięcie duplikatów
titanic_clear_columns.duplicated(keep='first')  ## sprawdzenie

PassengerId
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
12      False
13      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
25      False
26      False
27      False
28      False
29      False
30      False
31      False
32      False
33      False
34      False
35      False
36      False
37      False
38      False
39      False
40      False
41      False
42      False
43      False
44      False
45      False
46      False
47      False
48      False
49      False
50      False
51      False
52      False
53      False
54      False
55      False
56      False
57      False
58      False
59      False
60      False
61      False
62      False
63      False
64      False
65      False
66      False
67      False
68      False
69      False
70      False
71      False
72      False
73      False
74      

Na tym etapie zbiór jest oczyszczony z wartości niepoprawnych, uzupełniony o brakujące wartości i przygotowany do dalszej analizy. <br>
Ze względu na fakt, że pozostały nam jeszcze kolumny **Parch** oraz **SibSp**, które mogą dostarczyć nam ciekawych dodatkowych informacji na temat pasażera (czy podróżował sam, czy z rodziną). <br>
Postanowiłam więc dodać kolumnę "ALONE", która przechowywać będzie wartośći "With Family" lub "Without Family". Jest to dodatkowa cecha pasażera, która może przydać się w przyszłej analizie danych. 

In [248]:
titanic_clear_columns['Alone'] = titanic_clear_columns.Parch + titanic_clear_columns.SibSp
titanic_clear_columns['Alone'].loc[titanic_clear_columns['Alone']>0] = 'With Family'
titanic_clear_columns['Alone'].loc[titanic_clear_columns['Alone'] == 0] = 'Without Family'
titanic_clear_columns.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Alone
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,"7,25",NaN,S,With Family
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,"71,2833",C85,C,With Family
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,"7,925",NaN,S,Without Family
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,"53,1",C123,S,With Family
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,"8,05",NaN,S,Without Family


Jak już wcześniej wspomninałam kolumna **Age** może być też pomocna do uzyskania dodatkowej cechy pasażera, mianowicie określenie czy dany pasażer był dzieckiem, czy nie. 
Dodano więc kolumne **Person**, która będzie przechowywać wartości *child* jeśli dany pasażer był dzieckiem, w innym przypadku kolumna zawiera płeć z kolumny **Sex**

In [226]:
def titanic_children(passenger):    
    age , sex = passenger
    if age <16:
        return 'child'
    else:
        return sex
titanic_clear_columns['person'] = titanic_clear_columns[['Age','Sex']].apply(titanic_children,axis=1)
titanic_clear_columns.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Alone,person
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,S,With Family,male
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.28,C,With Family,female
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.92,S,Without Family,female
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.10,S,With Family,female
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,S,Without Family,male


Po wykonaniu wszystkich powyższych kroków możemy przygotować i oczyścić każdy zestaw danych, w tym sprawozdaniu użyliśmy akurat zbioru danych o Titanicu, przygotowaliśmy niekompletne i błędne dane oraz wyciągnęliśmy ukrytą wiedzę, której być może wcześniej nie widzieliśmy.

Z tak przygotowanymi danymi możemy przejść do budowania modeli dla tego zbioru danych oraz wykorzystać model uczenia się maszynowego do przewidywania danej obserwacji.

<br>Ostatnim krokiem będzie **wyeksportowanie przygotowanego DataFrame do pliku o rozszerzeniu .tsv**

In [250]:
titanic_clear_columns.to_csv('filename.tsv', sep = '\t')